## 1. 불법 주정차 감시용 카메라

In [1]:
import os
import pandas as pd
import numpy as np
import re
import konlpy
import geopandas as gpd

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
os.chdir('../data')

In [3]:
df1=pd.read_csv('15.오산시_무인교통단속카메라.csv') #이 데이터는 모두 불법주정차 단속용 카메라임
df2=pd.read_csv("전국무인교통단속카메라표준데이터.csv", encoding='euc-kr') #이 데이터는 단속구분이 4인 카메라만 불법주정차 단속용
df2=df2[df2['시군구명']=='오산시'] #행정구역이 오산시가 아닌데 오산시 데이터로 들어간 게(ex: 화성시 오산면) 있어서 뽑아냄
df2=df2[df2['단속구분']=='4'].drop(['관리기관명','관리기관전화번호','제공기관코드','제공기관명'], axis=1)

In [4]:
df1.rename(columns={'설치위치_위도':'lat', '설치위치_경도':'lon'}, inplace=True) #변수명 변경
df2.rename(columns={'위도':'lat', '경도':'lon'}, inplace=True) #변수명 변경

In [5]:
parking_cctv=pd.concat([df1[['lat', 'lon']], df2[['lat','lon']]]).reset_index().drop(['index'], axis=1) #최종
parking_cctv.head()

,lat,lon
0,37.159313,127.059357
1,37.145409,127.067796
2,37.159198,127.062386
3,37.158559,127.061920
4,37.156216,127.077991


In [6]:
parking_cctv.to_csv("parking_cctv.csv", index=False, header=True)

In [7]:
parking=pd.read_csv('1.오산시_주정차단속(2018~2020).csv')
parking=parking[parking['행정구역'].apply(lambda x: x[0:7]=='경기도 오산시')]
parking.rename(columns={'단속위치_경도':'lon', '단속위치_위도':'lat'}, inplace=True) #위도, 경도 col name 변경
parking['geometry'] = list(zip(parking['lat'], parking['lon']))
parking.dropna(inplace=True)
parking.to_csv('parking.csv', index=False, header=True)

## 2. 과속 단속 카메라

### (1) 20.오산시_CCTV설치현황.csv

In [8]:
df2=pd.read_csv("20.오산시_CCTV설치현황.csv")
df2_speed_cam=df2[df2['CCTV 유형코드']=='E']
df2_speed_cam=df2_speed_cam.rename(columns={'설치위치_위도': '경도', '설치위치_경도': '위도'})
df2_speed_cam #위도, 경도가 바뀌어있어서 change

,CCTV 유형코드,설치위치_도로명주소,경도,위도
557,E,다온초,127.090748,37.150892
558,E,다온초,127.090748,37.150892
559,E,오산숲유치원,127.083997,37.148336
560,E,대호초,127.057906,37.159936
561,E,성산초,127.065567,37.149489
562,E,성호초,127.070001,37.149096
563,E,대원초,127.072728,37.131781
564,E,원일초,127.071237,37.139445
565,E,운천초,127.078048,37.156239
566,E,운산초,127.076209,37.152348


### (2) 경찰청 전국무인교통단속카메라표준데이터

In [9]:
df_police=pd.read_csv("전국무인교통단속카메라표준데이터.csv", encoding='euc-kr')
df_police=df_police[df_police['시군구명']=='오산시']

df_police_speed_cam=df_police[df_police['단속구분']=='02'].drop(['관리기관명','관리기관전화번호','제공기관코드','제공기관명'], axis=1)
df_police_speed_cam.head()

,무인교통단속카메라관리번호,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,설치장소,단속구분,제한속도,단속구간위치구분,과속단속구간길이,보호구역구분,설치년도,데이터기준일자
674,F7153,경기도,오산시,05,해당없음,동부대로,NaN,NaN,경기도 오산시 부산동 782,37.154907,127.081104,오산시 부산동 782 오산경찰서 앞 사거리(동탄->오산시청),02,50,NaN,NaN,NaN,NaN,2020-08-31
675,F7154,경기도,오산시,05,해당없음,성호대로,NaN,NaN,경기도 오산시 오산동 876-1,37.148878,127.068554,오산시 오산동 876-1 중원사거리 (남촌오거리→오산시청),02,60,NaN,NaN,NaN,NaN,2020-08-31
990,F6118,경기도,오산시,05,해당없음,동부대로,NaN,NaN,경기도 오산시 갈곶동 336-11,37.131664,127.078272,오산시 갈곶동 336-11 오산자이아파트입구 삼거리(평택->오산시청),02,60,NaN,NaN,NaN,NaN,2020-08-31
991,F6119,경기도,오산시,05,해당없음,수목원로,NaN,NaN,경기도 오산시 누읍동 367-3,37.141910,127.054744,오산시 누읍동 367-3 누읍어린이공원앞 사거리(가수동->남부순환로),02,60,NaN,NaN,NaN,NaN,2020-08-31
992,F6120,경기도,오산시,05,해당없음,문시로,NaN,NaN,경기도 오산시 외삼미동 413-4,37.189685,127.052957,오산시 외삼미동 413-4 외삼미동마을 앞 사거리(동탄->세교동),02,60,NaN,NaN,NaN,NaN,2020-08-31


In [10]:
# 공공데이터 개방표준 개정 (11차) 전문에 따르면,
# 소재지도로명주소가 필수이나, 특별한 경우 도로명주소를 빼고, 지번주소를 필히 입력하라는 조항이 있음. 
# 따라서 도로명주소 삭제
# 대신, 지번주소를 사용할 것임

#단속구분: 1)속도 2)신호 3)통행위반(ex 버스전용차로) 4) 불법주정차 5) 기타 

#단속구분이 속도인 건, 전부 고속도로.
#단속구분이 신호인 건, 전부 제한속도가 있음.
#이 둘을 합치면, 경찰청 데이터에서 제한속도가 있는 카메라만 뽑은 것과 같음.(df_police_speed_cam)
#그런데, 잘 생각해보면, 고속도로에서의 사고는 특별히 어린이 사고와 연관성이 있는 것은 아닐 것이므로
#고속도로에 설치된 '속도위반으로 구분된' 카메라는 무시하고,
#제한속도가 있는 카메라 중 단속구분이 신호인 카메라 17대만을 과속단속카메라로 보기로 하자.

In [11]:
df_speed_final=pd.concat([df2_speed_cam[['위도','경도']], df_police_speed_cam[['위도','경도']]]).reset_index().drop(['index'], axis=1)
df_speed_final.rename(columns={'위도':'lat',
                              '경도':'lon'}, inplace=True)
df_speed_final['geometry'] = list(zip(df_speed_final['lat'], df_speed_final['lon']))
df_speed_final.head()

,lat,lon,geometry
0,37.150892,127.090748,"(37.15089232, 127.09074779999999)"
1,37.150892,127.090748,"(37.15089232, 127.09074779999999)"
2,37.148336,127.083997,"(37.14833581, 127.083997)"
3,37.159936,127.057906,"(37.15993649, 127.057906)"
4,37.149489,127.065567,"(37.14948855, 127.06556699999999)"


In [12]:
df_speed_final.to_csv("overspeed_cam.csv", index=False, header=True)

## 3. 유동인구

In [13]:
floating=pd.read_csv('8.오산시_유동인구(2019).csv')
floating.head()

,STD_YM,lon,lat,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,TMST_06,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
0,201901,126.995145,37.176703,0.08,0.05,0.04,0.04,0.08,0.27,0.80,...,0.69,0.76,0.95,0.91,0.70,0.43,0.34,0.22,0.14,0.10
1,201901,126.995142,37.177153,0.06,0.04,0.03,0.03,0.06,0.19,0.56,...,0.54,0.58,0.69,0.66,0.51,0.33,0.26,0.17,0.11,0.08
2,201901,126.995714,37.175804,0.00,0.00,0.00,0.00,0.00,0.01,0.03,...,0.04,0.04,0.05,0.05,0.03,0.02,0.02,0.01,0.01,0.00
3,201901,126.995711,37.176254,0.05,0.03,0.02,0.03,0.05,0.19,0.56,...,0.48,0.53,0.65,0.63,0.48,0.29,0.23,0.15,0.10,0.07
4,201901,126.995708,37.176705,0.02,0.01,0.01,0.01,0.02,0.07,0.18,...,0.19,0.20,0.23,0.22,0.17,0.11,0.09,0.06,0.04,0.03


In [14]:
# 7 ~ 23시 유동인구만 사용
floating.drop(['STD_YM','TMST_00', 'TMST_01', 'TMST_02', 'TMST_03', 'TMST_04', 'TMST_05', 'TMST_06'], axis=1, inplace=True)
floating['sum']=floating.iloc[:,3:19].sum(axis=1)

# 위도,경도 별로 합한 후, 평균 냄
floating=floating.groupby(['lat','lon']).mean()
floating=floating.add_suffix('_mean').reset_index()
floating.rename(columns={'sum_mean':'mean'}, inplace=True)

# 위도, 경도, 평균 column만 사용
floating.drop(floating.columns[2:19], axis=1, inplace=True)
floating['geometry'] = list(zip(floating['lat'], floating['lon']))
floating.head()

,lat,lon,mean,geometry
0,37.125623,127.071472,53.367500,"(37.1256226537081, 127.071472079079)"
1,37.125629,127.073161,12.580833,"(37.1256287454303, 127.073160660635)"
2,37.125631,127.073724,12.777500,"(37.1256307706599, 127.073723521266)"
3,37.125633,127.074286,11.696667,"(37.1256327932173, 127.07428638195299)"
4,37.126071,127.070907,51.132500,"(37.1260713170185, 127.070906674399)"


In [15]:
floating.to_csv("floating_pop.csv", index=False, header=True)

## 4. 어린이 대상 학원 및 교습소 추출

In [16]:
df = pd.read_csv("30.오산시_학원_및_교습소_현황.csv")

# null 값 삭제
df = df[df.isna().sum(axis = 1) == 0] 

# 초등학교 학원이라 보기 힘든 것들 삭제
df = df[df["업종구분명"] != '평생직업교육학원']
df = df[~df["교습과정명"].apply(lambda x : "기사" in x) ]

def extract_noun(x):
    text_join = x
    filtered_content = re.sub(r'[^\d\s\w]', ' ', text_join)
    filtered_content = re.sub(r'[\d PABCDEFGHIJKLMNOPGRSTUVWXYZQ]', ' ', filtered_content)
    filtered_content = re.sub(r'[abcdefghijklmnopqrstuvwxyz*]', ' ', filtered_content)
    twitter = konlpy.tag.Twitter()
    twitter_morphs = twitter.pos(filtered_content)
    word = set([i[0] for i in twitter_morphs if i[1] == "Noun"])
    return word

df["교습과정명"] = df["교습과정명"].apply(extract_noun)

def check(x):
    if "초" in x or "초등" in x or "저학년" in x or "어린이" in x:
        return False
    try:
        for item in ["간호", "가구", "가죽", "기능사", "네일아트", "다이어트", "세무","실무","실비",
                     "정보처리","회계","회사", "고등", "중등", "중고", "중학생","고등학생", "중학교",
                     "고등학교", "기사" ,"성인","정보처리기사", "가죽공예","웹디자인" "화장품"]:
            if item in x:
                return True
        return False
    except:
        return False

df = df[~df["교습과정명"].apply(check)]

df = df.reset_index(drop = True)
df.to_csv("학원.csv",index = False)

## 5. 함수에 넣기 위한 geometry 전처리
### 횡단보도, 인도 데이터
- reproject 함수를 사용하기 위해

In [17]:
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from functools import partial
import pyproj
from shapely import ops

In [18]:
def reproject(geom, from_proj=None, to_proj=None):
    tfm = partial(pyproj.transform, pyproj.Proj(init=from_proj), pyproj.Proj(init=to_proj))
    return ops.transform(tfm, geom)

In [19]:
df = gpd.read_file('17.오산시_횡단보도.geojson',encoding="utf-8")
df["geometry"] = df["geometry"].apply(lambda x: reproject(x, 'EPSG:4326', 'EPSG:26944'))
df.to_file("횡단보도.json", driver="GeoJSON")

In [20]:
df = gpd.read_file('21.오산시_인도.geojson',encoding="utf-8")
df["geometry"] = df["geometry"].apply(lambda x: reproject(x, 'EPSG:4326', 'EPSG:26944'))
df.to_file("인도.json", driver="GeoJSON")

## 6. 어린이 거주인구 및 거주비율

In [21]:
data = gpd.read_file("5.오산시_연령별_거주인구격자(유소년).geojson", encoding = "utf-8")
data.val[data.val.isna()] = 0
data.to_file("child_count.geojson", driver="GeoJSON")

In [22]:
data = gpd.read_file("5.오산시_연령별_거주인구격자(유소년).geojson", encoding = "utf-8")
data.val[data.val.isna()] = 0
total_data = gpd.read_file("4.오산시_연령별_거주인구격자(총인구).geojson", encoding = "utf-8")
total_data.val[total_data.val.isna()] = 0
data["total"] = total_data.val
data.to_file("child_rate_count.geojson", driver="GeoJSON")

## 7. 초등학교, 어린이집, 유치원 개수

In [23]:
data = pd.read_csv("10.오산시_학교위치정보.csv", encoding = "utf-8")
data = data[data["학교구분"] == "초등학교"]
data.rename(columns = {"학교위치_경도":"시설위치_경도", "학교위치_위도" : "시설위치_위도"}, inplace = True)
data = pd.read_csv("13.오산시_어린이집_유치원현황.csv", encoding = "utf-8")
school = pd.read_csv("10.오산시_학교위치정보.csv", encoding = "utf-8")
kinder = pd.read_csv("13.오산시_어린이집_유치원현황.csv", encoding = "utf-8")
elem = school[school["학교구분"] == "초등학교"]
data = kinder[["시설구분", "시설위치_경도", "시설위치_위도"]]
data.loc[:,"시설구분"] = "유치원과 어린이집"
elem = elem[["학교구분","학교위치_경도", "학교위치_위도"]].rename(columns = {"학교위치_경도":"시설위치_경도", "학교위치_위도" : "시설위치_위도","학교구분":"시설구분"})
data = pd.concat([data, elem], axis = 0)
data.index = range(len(data))
data.to_csv("elem_kinder_count.csv")

## 8. 사고 수

In [24]:
data = pd.read_csv("2021_01_12_03_44_final.csv")
data.loc[:,'geometry'] = data.loc[:,['lat','lon']].apply(lambda x: (x.lat, x.lon),axis=1)
data.to_csv('accident_count.csv')

## 9. 혼잡빈도

In [25]:
road_gdf = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
road_to_geo = {}
for i in range(len(road_gdf)):
  road_to_geo[road_gdf.link_id[i]] = road_gdf.geometry[i]
chaos1_df = pd.read_csv('25.평일_전일_오산시_혼잡빈도강도_Level6.csv')
chaos1_df['geometry'] = chaos1_df.iloc[:,0].apply(lambda x: road_to_geo[str(x)[:-2]] if str(x)[:-2] in road_to_geo.keys() else False)
chaos1_df.columns = ['link_id','rank','length','name','sido','sigungo','eub','time_range','work/weekend','chaos1','geometry']
#to gdf
chaos1_gdf = gpd.GeoDataFrame(chaos1_df, crs='EPSG:4326')
chaos1_gdf.to_csv('chaos1.csv')

## 10. 과속방지턱

In [28]:
bump=pd.read_csv('18.오산시_과속방지턱표준데이터.csv')

bump.rename(columns={'설치위치_경도':'lon',
                    '설치위치_위도':'lat'}, inplace=True)
bump=bump.reset_index()
bump['geometry'] = list(zip(bump['lat'], bump['lon']))

bump.to_csv("bump.csv")